In [ ]:
__author__ = 'Mike Fitzpatrick <fitz@noao.edu>, Glenn Eychaner <geychaner@noao.edu>, Robert Nikutta <nikutta@noao.edu>, Alice Jacques <alice.jacques@noao.edu>'
__version__ = '20200901' # yyyymmdd
__datasets__ = ['sdss_dr14']
__keywords__ = ['vospace','fits']

# How to Use the Data Lab Public File Services

*Revised:  Sept 01, 2020*

### Table of contents
* [Summary](#summary)
* [Disclaimer & Attribution](#attribution)
* [Imports & setup](#import)
* [Listing another user's public file space](#listspace)
* [Listing all available public file spaces](#listall)
* [Example: using the **SDSS DR14** public file service](#exsdss)

<a class="anchor" id="summary"></a>
# Summary

Files in the virtual storage VOSpace are usually identified via the prefix "*vos://*". This shorthand identifier is resolved to a user's home directory of the storage space in the service.  

If the "*vos://*" prefix is instead the name of another user (e.g. "*geychaner://*"), and the remainder of the path grants public or group read/write access, then the other user's spaces will be accessed. Most user spaces have a "*/public*" directory to facilitate file sharing (e.g. '*geychaner://public/foo.fits*' will access the '*foo.fits*' file from user '*geychaner*'). Users can make any file (or directory) public by moving it to (or creating a link in) their "/public" directory.

**Public file services** are specially created areas where all files are world-readable, and are used for serving files from Data Lab datasets.

<a class="anchor" id="attribution"></a>
# Disclaimer & Attribution

If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: http://datalab.noao.edu/disclaimers.php

<a class="anchor" id="import"></a>
# Imports & setup

In [ ]:
# Make matplotlib plot inline
%matplotlib inline

# Standard DL imports, note we only need storeClient
from dl import storeClient as sc

# 3rd Party Imports
import io
import numpy as np
from matplotlib import pyplot as p
from astropy.io import fits

<a class="anchor" id="listspace"></a>
# Listing another user's public file space
We can view the contents of another user's public file space if we know their username.

In [ ]:
print(sc.ls ('demo00://public',format='short'))

<a class="anchor" id="listall"></a>
# Listing all available public file spaces

A 'file service' is a **public** vospace, readable by all users. The *sc.services()* function allows a user to list all the available file services.

In [ ]:
print(sc.services())

<a class="anchor" id="exsdss"></a>

# An example using the **SDSS DR14** public file service.

We can explore the contents of a particular public file space using the *sc.ls* function:  

In [ ]:
print(sc.ls('sdss_dr14://'))

We can delve deeper into a particular file by specifying the file name when we call the *sc.ls* function:

In [ ]:
print(sc.ls ('sdss_dr14://eboss'))

In [ ]:
print(sc.ls ('sdss_dr14://eboss/spectro/'))

In [ ]:
print(sc.ls ('sdss_dr14://eboss/spectro/redux'))

### Set base directory and plate number
We can set the base directory and plate number of the object we want to look at:

In [ ]:
base = 'sdss_dr14://eboss/spectro/redux/v5_10_0/'
plate = '3615'

### List all available FITS plate files in the plate directory
We can construct the VOSpace path to the plate directory:

In [ ]:
spPlate = base + plate + '/spPlate-' + plate
print(sc.ls (spPlate + '*.fits', format='short'))

### Pick a modified Julian date (mjd) and fiber
With the mjd, we can construct the VOSpace path to the plate file. We then verify the path construction.

In [ ]:
mjd = '56544'
fiber = 39

spfile = spPlate + '-' + mjd + '.fits'
print ('File: ' + spfile)
print (sc.ls (spfile))

### Now read the spectrum from the file and construct the wavelength array
From this file we will also retrieve its header, flux, and inverse variance. 

In [ ]:
try:
    with fits.open(sc.get(spfile, mode='fileobj')) as hdulist:
        hdr = hdulist[0].header
        flux = hdulist[0].data[fiber-1, :]
        ivar = hdulist[1].data[fiber-1, :]
except Exception as e:
    raise ValueError("Could not find spPlate file for plate={0:s}, mjd={1:s}!".format(plate, mjd))

loglam = hdr['COEFF0'] + hdr['COEFF1']*np.arange(hdr['NAXIS1'], dtype=flux.dtype)
wavelength = 10.0**loglam
print("Length of wavelength array =", len(wavelength))
print("Length of flux array =", len(flux))
print("Length of ivar array =", len(ivar))

### Make a plot of the spectrum
Here we plot the wavelength and ivar-corrected flux.

In [ ]:
fig = p.figure(dpi=100)
p.plot(wavelength, flux* (ivar > 0), 'k')
p.xlabel('Wavelength [$\mathrm{\AA}$]')
p.ylabel('Flux')